Скачать датасет, разделенный на train, valid: https://disk.yandex.ru/d/e6ni0ZV8M5dnEA 

1. Решаем задачу бинарной классификации: является ли клиент фродовым. Транзакции для каждого клиента имеют таргет 'Is Fraud?' и он будет одинкаовым для каждой транзакции клиента.
2. Необходимо провести feature engineering
3. Выбрать подходящую архитектуру нейронной сети.
4. Обучить сеть
5. Показать графики обучения и точности модели

In [48]:
import pandas as pd
import numpy as np
import pickle

In [2]:
data = pd.read_csv('/Users/macbook/Desktop/card_transaction.v1.csv')

In [3]:
data.columns.values

array(['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount',
       'Use Chip', 'Merchant Name', 'Merchant City', 'Merchant State',
       'Zip', 'MCC', 'Errors?', 'Is Fraud?'], dtype=object)

In [4]:
data

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020,2,27,22:23,$-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386896,1999,1,2020,2,27,22:24,$54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386897,1999,1,2020,2,28,07:43,$59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No
24386898,1999,1,2020,2,28,20:10,$43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No


In [51]:
len(train_data), len(valid_data), len(test_data)

(17397216, 3412698, 3576986)

In [52]:
len(train_data['User'].unique()), len(valid_data['User'].unique()), len(test_data['User'].unique())

(1400, 300, 300)

## Как использовать BatchNorm в RNN сети

1. Создаем класс, который наследуюется от BatchNorm

In [ ]:
class PackedSequenceBatchNorm1d(torch.nn.BatchNorm1d):
    def forward(self, x):

        return torch.nn.utils.rnn.PackedSequence(
            data=super().forward(x.data),
            batch_sizes=x.batch_sizes,
            sorted_indices=x.sorted_indices
        )

2. В конструктуре создаем объект данного класса

In [ ]:
self.feature_bn = PackedSequenceBatchNorm1d(num_features=self.nrof_features)

3. Вызываем его в forward с учетом padding последовательности транзакций

In [ ]:
n_output_features = nn.utils.rnn.pack_padded_sequence(output_features.to(self.device), origin_nrof_trans,
                                                enforce_sorted=False, batch_first=True)
bn_output_features = self.feature_bn(bn_output_features)
bn_output_features, _ = nn.utils.rnn.pad_packed_sequence(bn_output_features, batch_first=True)